In [24]:
# experiment_notebook.ipynb

# Step 1: Load an experiment configuration from a JSON file
import json
from scripts.config_model import Experiment
from scripts.experiment_loader import load_experiment_config, save_experiment_config
from scripts.sample_db import SampleDB

json_file_path = '../data/calcium/2022-04-26/f3/sampleConfig_20220426_RM0008_130hpf_fP1_f3.json'

config = load_experiment_config(json_file_path)
print("Loaded experiment configuration:")
print(config)

Loaded experiment configuration:
params_odor=ParamsOdor(odor_list=['Ala', 'Ser', 'Ctrl', 'TCA', 'GCA', 'TDCA', 'Cad', 'SA'], odor_sequence=['Ala', 'Ser', 'Ctrl', 'TCA', 'GCA', 'TDCA', 'Cad', 'SA', 'Cad', 'TDCA', 'GCA', 'TCA', 'Ser', 'Ctrl', 'Ala', 'SA', 'Ctrl', 'Cad', 'Ala', 'Ser', 'TDCA', 'GCA', 'TCA', 'SA'], odor_concentration_uM=[OdorConcentration(name='Ala', concentration_mM=100.0), OdorConcentration(name='Ser', concentration_mM=100.0), OdorConcentration(name='Ctrl', concentration_mM=100.0), OdorConcentration(name='TCA', concentration_mM=10.0), OdorConcentration(name='GCA', concentration_mM=10.0), OdorConcentration(name='TDCA', concentration_mM=10.0), OdorConcentration(name='Cad', concentration_mM=300.0), OdorConcentration(name='SA', concentration_mM=0.0)], n_trials=3, n_frames=1500, pulse_delay_s=15, pulse_duration_s=5, trial_interval_s=60, missed_trials=[], events=[]) params_lm=ParamsLM(start_time=datetime.datetime(2022, 4, 26, 19, 29), end_time=datetime.datetime(2022, 4, 26, 21,

In [20]:
# Step 2: Add the experiment to a sample database
sample_db = SampleDB()
sample_db_path = '../data/sample_db.json'
sample_db.load(sample_db_path)
print("Loaded existing sample database.")

sample = config.sample
existing_sample = sample_db.get_sample(sample.id)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [11]:
# Step 3: Check if the sample is already in the database and ask if you want to update it
if existing_sample:
    print(f"Sample with ID {sample.id} already exists.")
    update = input("Do you want to update the existing sample? (yes/no): ").strip().lower() == 'yes'
    raw_path = input("Enter the raw path: ")
    anatomy_path = input("Enter the anatomy path: ")
    em_path = input("Enter the EM path (optional): ").strip() or None
    sample_db.add_or_update_sample(sample, raw_path, anatomy_path, em_path, update=update)
else:
    raw_path = input("Enter the raw path: ")
    anatomy_path = input("Enter the anatomy path: ")
    em_path = input("Enter the EM path (optional): ").strip() or None
    sample_db.add_or_update_sample(sample, raw_path, anatomy_path, em_path)

# Step 4: Save the updated database to a file
sample_db.save(csv_file_path)
print(f"Sample database saved to {csv_file_path}.")

# Step 5: Save the updated configuration back to the JSON file if needed
save_config = input("Do you want to save the updated configuration back to the JSON file? (yes/no): ").strip().lower() == 'yes'
if save_config:
    save_experiment_config(config, json_file_path)
    print(f"Experiment configuration saved to {json_file_path}.")

NameError: name 'existing_sample' is not defined

In [12]:
# Step 4: Save the updated database to a file
sample_db.save(db_file_path)
print(f"Sample database saved to {db_file_path}.")

FileNotFoundError: [Errno 2] No such file or directory: 'data/sample_db.json'

In [14]:
# Step 5: Save the updated configuration back to the JSON file if needed
save_config = input("Do you want to save the updated configuration back to the JSON file? (y/n): ").strip().lower() == 'y'
if save_config:
    save_experiment_config(config, json_file_path)
    print(f"Experiment configuration saved to {json_file_path}.")
